# Mitigación de sesgo en el modelo original

## Índice de Contenido

1. [Preparación del entorno y carga de datos](#1-Preparación-del-entorno-y-carga-de-datos)
2. [Primer método de mitigación](#2-Primer-método-de-mitigación)
    - [Aplicación del método y entrenamiento del modelo](#2.1-Aplicación-del-método-y-entrenamiento-del-modelo)
    - [Evaluación del modelo mediante métricas de clasificación](#2.2-Evaluación-del-modelo-mediante-métricas-de-clasificación)
3. [Segundo método de mitigación](#3-Segundo-método-de-mitigación)
    - [Aplicación del método y entrenamiento del modelo](#3.1-Aplicación-del-método-y-entrenamiento-del-modelo)
    - [Evaluación del modelo mediante métricas de clasificación](#3.2-Evaluación-del-modelo-mediante-métricas-de-clasificación)
4. [Evaluación de equidad de modelos mitigados](#4-Evaluación-de-equidad-de-modelos-mitigados)
5. [Resumen y conclusiones](#5-Resumen-y-conclusiones)

### Si tomamos como umbral 0.1 se dejaria de cumplir Equalized Odds, por lo que intentaremos mitigar la metrica de fpr.

# 1. Preparación del entorno y carga de datos

In [162]:
import pandas as pd
import os 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Carga de datos
ruta_archivo = os.path.join('..', 'data', 'processed', 'df_genero_procesado.csv')
g_credit_data = pd.read_csv(ruta_archivo)

# 2. Primer método de mitigación

#### Como el primer metodo de mitigacion será pre-processing , debemos utilizar nuevamente los datos previos al entrenamiento de cualquier modelo

## 2.1 Aplicación del método y entrenamiento del modelo

In [163]:
# Separamos variables predictoras (X) y variable objetivo (y)

X = g_credit_data.drop(columns=['target'])
y = g_credit_data['target']
indices = X.index

# Dividimos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X.values, y.values, indices, test_size=0.2, random_state=42
)

X_test = pd.DataFrame(X_test, columns=X.columns, index=idx_test)
y_test = pd.Series(y_test, name='target', index=idx_test)
X_train = pd.DataFrame(X_train, columns=X.columns, index=idx_train)
y_train = pd.Series(y_train, name='target', index=idx_train)

print(f"Dimensiones de X_train_design: {X_train_design.shape}")
print(f"Dimensiones de X_test_design: {X_test_design.shape}")
print(f"Distribución de clases en el conjunto de entrenamiento: {np.bincount(y_train)}")
print(f"Distribución de clases en el conjunto de prueba: {np.bincount(y_test)}")

Dimensiones de X_train_design: (800, 18)
Dimensiones de X_test_design: (200, 18)
Distribución de clases en el conjunto de entrenamiento: [241 559]
Distribución de clases en el conjunto de prueba: [ 59 141]


#### Se ejecuta el Baseline con modelo LogisticRegressor de Scikit-learn

In [164]:
from holisticai.bias.metrics import classification_bias_metrics
from sklearn.preprocessing import StandardScaler
from holisticai.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Normalizamos los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

modelo = LogisticRegression(random_state=42, max_iter=1000)
modelo.fit(X_train_scaled, y_train)

y_pred_test = modelo.predict(X_test_scaled)

In [165]:
metrics = {}

baseline = classification_bias_metrics(X_test['genre_male']==1, X_test['genre_male']==0, y_pred_test, y_test, metric_type='both')

baseline_accuracy = accuracy_score(y_test, y_pred_test)

metrics['Baseline'] = baseline
print(baseline)
print(f"Baseline accuracy: {baseline_accuracy:.4f}")

                                       Value  Reference
Metric                                                 
Statistical Parity                  0.083333          0
Disparate Impact                    1.111111          1
Four Fifths Rule                    0.900000          1
Cohen D                             0.213470          0
2SD Rule                            1.348838          0
Equality of Opportunity Difference  0.049774          0
False Positive Rate Difference      0.148459          0
Average Odds Difference             0.099117          0
Accuracy Difference                -0.003968          0
Baseline accuracy: 0.7650


#### Notamos que aun ajustando un regresor de Scikit-learn sigue dando una diferencia en fpr mayor a 0.1.
#### Utilizamos el metodo pre-processing CorrelationRemover

In [166]:
from holisticai.bias.mitigation import CorrelationRemover 
mitigator = CorrelationRemover()
model = LogisticRegression(random_state=42, max_iter=1000)

pipeline = Pipeline(steps=[('scalar', StandardScaler()), ("bm_preprocessing", mitigator), ("estimator", model),])
pipeline.fit(X_train, y_train, bm__group_a=X_train['genre_male'] == 1, bm__group_b=X_train['genre_male'] == 0)

y_pred_pipeline = pipeline.predict(X_test, bm__group_a=X_test['genre_male'] == 1, bm__group_b=X_test['genre_male'] == 0)

metrics_preprocessing_correlationRemover = classification_bias_metrics(X_test['genre_male']==1, X_test['genre_male']==0, y_pred_pipeline, y_test, metric_type='both')

accuracy_preprocessing_correlationRemover = accuracy_score(y_test, y_pred_pipeline)

metrics['CorrelationRemover'] = metrics_preprocessing_correlationRemover
print(metrics_preprocessing_correlationRemover)
print(f"Preprocessing model accuracy: {accuracy_preprocessing_correlationRemover:.4f}")

                                       Value  Reference
Metric                                                 
Statistical Parity                 -0.097222          0
Disparate Impact                    0.888889          1
Four Fifths Rule                    0.888889          1
Cohen D                            -0.246777          0
2SD Rule                           -1.558155          0
Equality of Opportunity Difference -0.056561          0
False Positive Rate Difference     -0.205882          0
Average Odds Difference            -0.131222          0
Accuracy Difference                 0.027778          0
Preprocessing model accuracy: 0.7700


## 2.2 Evaluación del modelo mediante métricas de clasificación

#### El modelo empeoró su performance con el fpr.

# 3. Segundo método de mitigación

### Utilizamos el metodo Inprocessing PrejudiceRemover

## 3.1 Aplicación del método y entrenamiento del modelo

In [167]:
# Define inprocessing model
from holisticai.bias.mitigation import PrejudiceRemover

model = LogisticRegression(random_state=42, max_iter=1000)
mitigator = PrejudiceRemover(maxiter=100, fit_intercept=True, verbose=1, print_interval=1).transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
group_a_train = X_train['genre_male'] == 1
group_b_train = X_train['genre_male'] == 0
group_a_test = X_test['genre_male'] == 1
group_b_test = X_test['genre_male'] == 0
X_train = scaler.fit_transform(X_train)
mitigator.fit(X_train, y_train, group_a_train, group_b_train)

# Standardize data and predict
X_test = scaler.transform(X_test)
y_pred = mitigator.predict(X_test, group_a_test, group_b_test)

# Evaluate bias metrics
metrics_prejudice = classification_bias_metrics(group_a_test, group_b_test, y_pred, y_test, metric_type='both')
metrics['PrejudiceRemover'] = metrics_prejudice
print(metrics['PrejudiceRemover'])
print(f"Inprocessing model accuracy: {accuracy_score(y_test, y_pred):.4f}")

[elapsed time: 00:00:00 | iter:6/100 | loss:399.3788]
[elapsed time: 00:00:00 | Best Loss : 399.3788]
                                       Value  Reference
Metric                                                 
Statistical Parity                 -0.033730          0
Disparate Impact                    0.959811          1
Four Fifths Rule                    0.959811          1
Cohen D                            -0.086919          0
2SD Rule                           -0.551587          0
Equality of Opportunity Difference -0.046757          0
False Positive Rate Difference     -0.016807          0
Average Odds Difference            -0.031782          0
Accuracy Difference                -0.021825          0
Inprocessing model accuracy: 0.7700


## 3.2 Evaluación del modelo mediante métricas de clasificación

### El modelo mejoró notablemente su diferencia en fpr.

# 4. Evaluación de equidad de modelos mitigados

In [168]:
# Comparison table of bias metrics

keys = ['Baseline', 'CorrelationRemover', 'PrejudiceRemover', 'Reference']
comparison = pd.concat([baseline['Value'], metrics_preprocessing_correlationRemover['Value'], metrics_prejudice], axis=1)
comparison.columns = keys

def highlight_closest(s):
    reference = s['Reference']
    differences = s.drop('Reference').apply(lambda x: abs(x - reference))
    closest = differences.idxmin()
    return ['background-color: mediumseagreen' if x == s[closest] else '' for x in s]

comparison_highlighted = comparison.style.apply(highlight_closest, axis=1)
comparison_highlighted

,Baseline,CorrelationRemover,PrejudiceRemover,Reference
Metric,,,,
Statistical Parity,0.083333,-0.097222,-0.033730,0
Disparate Impact,1.111111,0.888889,0.959811,1
Four Fifths Rule,0.900000,0.888889,0.959811,1
Cohen D,0.213470,-0.246777,-0.086919,0
2SD Rule,1.348838,-1.558155,-0.551587,0
Equality of Opportunity Difference,0.049774,-0.056561,-0.046757,0
False Positive Rate Difference,0.148459,-0.205882,-0.016807,0
Average Odds Difference,0.099117,-0.131222,-0.031782,0
Accuracy Difference,-0.003968,0.027778,-0.021825,0


# 5. Resumen y conclusiones

#### El modelo mitigado con PrejudiceRemover es un modelo que cumple mejora casi en su totalidad las metricas que miden fairness